In [1]:
#
# Atividade de Aprofundamento 
# SVM Classification with PCA
#
# Tarefa. SVM Classification com PCA
#
# Nesta atividade voc� vai classificar os dados breast-cancer-wisconsin com 
# uma SVM (modelo tradicional de aprendizado supervisionado) para,
# na sequencia classificar, os dados reduzidos a componentes principais (PCA)
# e comparar os resultados obtidos.
#

#--------------------------------------------------------------------------
#
# Empregue esse c�digo como modelo. Preencha e execute conforme as instru��es e responda
# as quest�es correspondentes. Em alguns pontos verifique as instru��es de check para que
# voc� n�o leve erros muito adiante.
#
#--------------------------------------------------------------------------

#
# instale e importe as bibliotecas necess�rias 
#
install.packages("e1071")
library(e1071)
library(ggplot2) 

#
# Mantenha o seed para correta resposta dos exerc�cios
#
RNGversion("3.5.2")
set.seed(1987)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Warning message in RNGkind("Mersenne-Twister", "Inversion", "Rounding"):
“non-uniform 'Rounding' sampler used”


In [2]:
#
# read breast-cancer-wisconsin
#
wdbc = read.csv(
  "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data",
  col.names=c(
    "patientid",
    "outcome",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concavepoints_mean",
    "symmetry_mean",
    "fractaldimension_mean",
    "radius_error",
    "texture_error",
    "perimeter_error",
    "area_error",
    "smoothness_error",
    "compactness_error",
    "concavity_error",
    "concavepoints_error",
    "symmetry_error",
    "fractaldimension_error",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concavepoints_worst",
    "symmetry_worst",
    "fractaldimension_worst"))

# Atualizado em 2021-06-07
wdbc = as.data.frame(unclass(wdbc),stringsAsFactors=T)

In [3]:
head(wdbc)

summary(wdbc)

,patientid,outcome,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concavepoints_mean,⋯,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concavepoints_worst,symmetry_worst,fractaldimension_worst
,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,⋯,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,⋯,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,⋯,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,⋯,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,⋯,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.1127,0.07400,⋯,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368


   patientid         outcome  radius_mean      texture_mean   perimeter_mean  
 Min.   :     8670   B:357   Min.   : 6.981   Min.   : 9.71   Min.   : 43.79  
 1st Qu.:   869222   M:211   1st Qu.:11.697   1st Qu.:16.18   1st Qu.: 75.14  
 Median :   906157           Median :13.355   Median :18.86   Median : 86.21  
 Mean   : 30423820           Mean   :14.120   Mean   :19.31   Mean   : 91.91  
 3rd Qu.:  8825022           3rd Qu.:15.780   3rd Qu.:21.80   3rd Qu.:103.88  
 Max.   :911320502           Max.   :28.110   Max.   :39.28   Max.   :188.50  
   area_mean      smoothness_mean   compactness_mean  concavity_mean   
 Min.   : 143.5   Min.   :0.05263   Min.   :0.01938   Min.   :0.00000  
 1st Qu.: 420.2   1st Qu.:0.08629   1st Qu.:0.06481   1st Qu.:0.02954  
 Median : 548.8   Median :0.09587   Median :0.09252   Median :0.06140  
 Mean   : 654.3   Mean   :0.09632   Mean   :0.10404   Mean   :0.08843  
 3rd Qu.: 782.6   3rd Qu.:0.10530   3rd Qu.:0.13040   3rd Qu.:0.12965  
 Max.   :2501.0

In [12]:
# Elimine portanto o atritbuto que n�o faz parte do treinamento   
#

my_breast = wdbc[,!(names(wdbc) %in% c("patientid"))]

nrow(my_breast)

[1] 568

In [23]:
# Construa os dados de Treinamento e Teste (Teste com 30% dos dados (round))

T <- sample(1:nrow(my_breast),round(nrow(my_breast)/3.333))

my_breast_train <- my_breast[-T,]
my_breast_test <- my_breast[T,]

ncol(my_breast_train)
nrow(my_breast_train)
ncol(my_breast_test)
nrow(my_breast_test)

[1] 31

[1] 398

[1] 31

[1] 170

In [24]:
#
# Fa�a o Treinamento com o conjunto de dados de treinamento
#

#
# SVM Radial 
#

svm_radial <- svm(outcome ~ ., data = my_breast_train, 
           scale = TRUE, kernel ="radial")

#
# Explore a SVM constru�da

summary(svm_radial)


Call:
svm(formula = outcome ~ ., data = my_breast_train, kernel = "radial", 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  96

 ( 49 47 )


Number of Classes:  2 

Levels: 
 B M




In [28]:
#
# Q3 Quantos vetores de suporte para a SVM radial e qual o grau do polinomio do kernel? (check)

#
# SVM Linear
#

svm_linear <- svm(outcome ~ ., data = my_breast_train, 
           scale = TRUE, kernel ="polynomial")

#
# Explore a SVM constru�da

summary(svm_linear)


Call:
svm(formula = outcome ~ ., data = my_breast_train, kernel = "polynomial", 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  polynomial 
       cost:  1 
     degree:  3 
     coef.0:  0 

Number of Support Vectors:  135

 ( 68 67 )


Number of Classes:  2 

Levels: 
 B M




In [29]:
#
# SVM sigmoid
#

svm_sigmoid <- svm(outcome ~ ., data = my_breast_train, 
           scale = TRUE, kernel ="sigmoid")


#
# Explore a SVM constru�da


summary(svm_sigmoid)




Call:
svm(formula = outcome ~ ., data = my_breast_train, kernel = "sigmoid", 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  sigmoid 
       cost:  1 
     coef.0:  0 

Number of Support Vectors:  61

 ( 31 30 )


Number of Classes:  2 

Levels: 
 B M




In [30]:
#
# SVM sigmoid
#

svm_linear <- svm(outcome ~ ., data = my_breast_train, 
           scale = TRUE, kernel ="linear")


#
# Explore a SVM constru�da


summary(svm_linear)




Call:
svm(formula = outcome ~ ., data = my_breast_train, kernel = "linear", 
    scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  32

 ( 16 16 )


Number of Classes:  2 

Levels: 
 B M




In [59]:
#
# Fa�a a predi��o do conjunto de dados de teste 
#
#
# Q5 Qual a acuracidade da SVM na classifica��o dos tumores quando 
# aplicada diretamente aos dados para cada um dos kernels? 
#



svm = svm(outcome ~ ., data = my_breast_test, 
           scale = TRUE, kernel = "radial") 
           # substitua aqui pela svm do kernel desejado e execute a predi��o 

             
predict_test = predict(svm, my_breast_test) 

c_matrix=table(my_breast_test$outcome, predict_test)
print(c_matrix)

cat('Accuracy kernel type (', svm$kernel, ") =", sum(diag(c_matrix))/sum(c_matrix)*100, ' %')

   predict_test
      B   M
  B 117   0
  M   1  52
Accuracy kernel type ( 2 ) = 99.41176  %

### Parte 2

In [49]:
#
# Parte b. Classificando os dados ap�s aplicar o PCA 
#

#
# Agora, vamos transformar a base my_breast, em uma base com atributos 
# de componente principais e, ent�o, aplicar o classificador SVM para
# a um conjunto de atributos reduzido para compararmos os resultados
#

#
# Passo 1. � transformar o dataframe my_breast em um dataframe de componentes
# principais my_breast_col_pc (com as colunas de componentes principais)
#

#
# Construindo my_breast em PC

# 
# Voc� pode empregar princomp ou principal. A entrada tem de eliminar 
# o atributo "outcome". Ele � n�o num�rico e a classe de treinamento, 
# n�o � portanto um atributo para redu��o a componentes principais
# 
# Consulte o help() dos comandos
#

breast_pc = princomp( my_breast[,-1] , cor=TRUE)
# breast_pc = principal(my_breast[,-1], nfactors=......)

#
# Explore a reconstru��o em PCs

# biplot(breast_pc) # opcional
names(breast_pc)
summary(breast_pc) 

#
# Q6 Quantos componentes s�o necess�rios para se ter uma representa��o com MAIS
# de 75% da variancia dos dados? E com MAIS 90% da variancia dos dados?   

[1] "sdev"     "loadings" "center"   "scale"    "n.obs"    "scores"   "call"

Importance of components:
                         Comp.1    Comp.2    Comp.3    Comp.4     Comp.5
Standard deviation     3.638981 2.3910098 1.6829215 1.4025409 1.28591886
Proportion of Variance 0.441406 0.1905643 0.0944075 0.0655707 0.05511958
Cumulative Proportion  0.441406 0.6319703 0.7263778 0.7919485 0.84706806
                           Comp.6     Comp.7    Comp.8     Comp.9    Comp.10
Standard deviation     1.10053051 0.81944608 0.6913661 0.64732389 0.59272739
Proportion of Variance 0.04037225 0.02238306 0.0159329 0.01396761 0.01171086
Cumulative Proportion  0.88744030 0.90982337 0.9257563 0.93972388 0.95143474
                           Comp.11     Comp.12     Comp.13     Comp.14
Standard deviation     0.543614149 0.511029351 0.492912107 0.396266161
Proportion of Variance 0.009850545 0.008705033 0.008098745 0.005234229
Cumulative Proportion  0.961285281 0.969990315 0.978089059 0.983323288
                           Comp.15     Comp.16     Comp.17     Comp.18
Standard deviation 

In [50]:
#
# Os scores s�o os novos atributos com base em componentes principais.
# Construa agora a base com os componentes principais, adicionado o atributo classe 
#

breast_col_pc = data.frame(breast_pc$scores) 

#
# vamos readicionar o aributo classe "outcome" 

breast_col_pc = cbind(wdbc$outcome,breast_col_pc)
names(breast_col_pc)[1] = "outcome"

#
# Explore my_breast e breast_col_pc, a constru�da com PCs 
#
# S�o os mesmos conjuntos de dados, mas a base em PCs tem os dados rotacionados  
#

#
# Note que as bases tem dados completamente diferentes. my_breast, os dados originais
# e breast_col_pc com os atributos que s�o os componentes principais

In [51]:
summary(breast_col_pc)

 outcome     Comp.1           Comp.2             Comp.3        
 B:357   Min.   :-5.559   Min.   :-12.6007   Min.   :-10.2108  
 M:211   1st Qu.:-2.754   1st Qu.: -1.2196   1st Qu.: -0.8423  
         Median :-1.172   Median :  0.1938   Median :  0.1513  
         Mean   : 0.000   Mean   :  0.0000   Mean   :  0.0000  
         3rd Qu.: 2.523   3rd Qu.:  1.5033   3rd Qu.:  1.0121  
         Max.   :16.409   Max.   :  7.7646   Max.   :  4.8976  
     Comp.4             Comp.5            Comp.6             Comp.7       
 Min.   :-5.21160   Min.   :-4.7665   Min.   :-4.15490   Min.   :-5.5143  
 1st Qu.:-0.79859   1st Qu.:-0.6417   1st Qu.:-0.73610   1st Qu.:-0.4577  
 Median : 0.09338   Median :-0.0240   Median :-0.06768   Median : 0.0201  
 Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.0000  
 3rd Qu.: 0.98326   3rd Qu.: 0.5995   3rd Qu.: 0.62419   3rd Qu.: 0.4493  
 Max.   : 5.01765   Max.   : 7.4394   Max.   : 5.17155   Max.   : 4.1038  
     Comp.8             Com

In [61]:
ncol(breast_col_pc)

[1] 31

In [52]:
# Note que as bases tem dados completamente diferentes. my_breast, os dados originais
# e breast_col_pc com os atributos que s�o os componentes principais
#
# Q7 Quantos componentes principais foram gerados?  
# Q8 Considere os 3 primeiros atributos de cada conjunto de dados (o original e o 
# de componentes principais) e a matriz de correla��o de cada conjunto. Quais as 
# correla��es observadas?
#

head(my_breast[,1:4])
cor(my_breast[,2:4])

head(breast_col_pc[,1:4])
cor(breast_col_pc[,2:4])

,outcome,radius_mean,texture_mean,perimeter_mean
,<fct>,<dbl>,<dbl>,<dbl>
1,M,20.57,17.77,132.90
2,M,19.69,21.25,130.00
3,M,11.42,20.38,77.58
4,M,20.29,14.34,135.10
5,M,12.45,15.70,82.57
6,M,18.25,19.98,119.60


,radius_mean,texture_mean,perimeter_mean
radius_mean,1.0000000,0.3293830,0.9978764
texture_mean,0.3293830,1.0000000,0.3359176
perimeter_mean,0.9978764,0.3359176,1.0000000


,outcome,Comp.1,Comp.2,Comp.3
,<fct>,<dbl>,<dbl>,<dbl>
1,M,2.416345,3.763969,0.5328256
2,M,5.768399,1.051489,0.5576766
3,M,7.111801,-10.335227,3.2268780
4,M,3.961135,1.946549,-1.4099223
5,M,2.381271,-3.975577,2.9245524
6,M,2.269394,2.678048,1.6515918


,Comp.1,Comp.2,Comp.3
Comp.1,1.000000e+00,-7.214026e-16,-1.703380e-17
Comp.2,-7.214026e-16,1.000000e+00,-3.713165e-17
Comp.3,-1.703380e-17,-3.713165e-17,1.000000e+00


In [54]:
# Passo 2. � classificar os dados com base nos Componentes Principais
#
# Vamos primeiramente empregar 3 componentes, em seguida 7 

# 
# Construindo os conjuntos de treinamento e teste...

# Empregar a mesma sele��o de dados T empregada anteriormente 
#

T <- sample(1:nrow(breast_col_pc),round(nrow(breast_col_pc)/3.333))


my_breast_col_pc_test <- breast_col_pc[T,] 
my_breast_col_pc_train <- breast_col_pc[-T,]


# Treinamento da SVM com os dados de treinamento para 3 componentes 
# ...[,c(1:4)] = "outcome","Comp.1","Comp.2","Comp.3"

In [57]:
# Vamos empregar somente o kernel com base radial

svm_3 <- svm(outcome ~ ., data = my_breast_col_pc_train[,c(1:4)], 
           scale = TRUE, kernel ="radial")

# Explore o modelo obtido
#

# check, 97 vetores de suporte devem ser formados 
#
print(svm_3)
summary(svm_3)

# Predi��o do conjunto de testes 
#

predict_test = predict(svm_3, my_breast_col_pc_test) 

c_matrix=table(my_breast_col_pc_test$outcome, predict_test)
print(c_matrix)

cat('Accuracy (3 PCs): ', sum(diag(c_matrix))/sum(c_matrix)*100, ' %')


Call:
svm(formula = outcome ~ ., data = my_breast_col_pc_train[, c(1:4)], 
    kernel = "radial", scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  92




Call:
svm(formula = outcome ~ ., data = my_breast_col_pc_train[, c(1:4)], 
    kernel = "radial", scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  92

 ( 47 45 )


Number of Classes:  2 

Levels: 
 B M




   predict_test
      B   M
  B 108   2
  M   6  54
Accuracy (3 PCs):  95.29412  %

In [58]:
# Treinamento da SVM com os dados de treinamento para 7 componentes 
# ...[,c(1:8)] = "outcome","Comp.1","Comp.2",...,"Comp.7"
#
# Vamos empregar somente o kernel com base radial

svm_7 <- svm(outcome ~ ., data = my_breast_col_pc_train[,c(1:8)] , 
             scale = TRUE, kernel ="radial")

# Explore o modelo obtido
#

# check, 130 vetores de suporte devem ser formados 
#

summary(svm_7)

# Predi��o do conjunto de testes 
#

predict_test = predict(svm_7, my_breast_col_pc_test) 

c_matrix=table(my_breast_col_pc_test$outcome, predict_test)
print(c_matrix)


cat('Accuracy (7 PCs): ', sum(diag(c_matrix))/sum(c_matrix)*100, ' %')


Call:
svm(formula = outcome ~ ., data = my_breast_col_pc_train[, c(1:8)], 
    kernel = "radial", scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  126

 ( 62 64 )


Number of Classes:  2 

Levels: 
 B M




   predict_test
      B   M
  B 108   2
  M   3  57
Accuracy (7 PCs):  97.05882  %